<a href="https://colab.research.google.com/github/lukmanr/codenext/blob/master/Training_Loop_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
# weights W[784, 10]   784=28*28
W = tf.Variable(tf.zeros([784, 10]))

# biases b[10]
b = tf.Variable(tf.zeros([10]))

In [0]:
# load the MNIST data set. The training set and test set are split
# automatically. 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
# convert the integer pixel values to floats
x_train = x_train / 255.0
x_test = x_test / 255.0

# reshape the images to be 2-D tensors of 1 x 784 pixels
x_train = x_train.reshape([-1, 1, 784])
x_test = x_test.reshape([-1, 1, 784])

In [0]:
# The neural network
def neural_network(X, W, b):
  Inputs = tf.matmul(X, W) + b
  Y = tf.nn.softmax(Inputs)
  return Y

In [0]:
# The loss function
def loss(Y, Y_l):
  return tf.reduce_sum(tf.square(Y - Y_l))

In [0]:
num_epochs = 1
learning_rate = 0.01

# the outer training loop:  repeat for num_epochs
for e in range(num_epochs):

    # the inner training loop: train on one image and label from the data set
    for image, label in zip(x_train, y_train):

        # convert the image and label to tensors
        X = tf.Variable(image, dtype=tf.float32)
        Y_l = tf.Variable(label, dtype=tf.float32)        

        # we wrap this 'with' statement around the next two lines, to tell 
        # TensorFlow to auto-compute the gradients
        with tf.GradientTape() as tape:
            # now get the output of the neural net
            Y = neural_network(X, W, b)

            # compute the loss function 
            current_loss = loss(Y, Y_l)

        # compute the gradients of the weights and biases with respect to the
        # loss function
        dW, db = tape.gradient(current_loss, [W, b])

        # update the weights and biases, by multiplying the gradients by the
        # learning rate
        W.assign(W + learning_rate * dW)
        b.assign(b + learning_rate * db)

        print(current_loss.numpy())